In [1]:
from deephaven_server import Server
from pydeephaven import Session

# Запуск сервера Deephaven
#s = Server(port=8080, jvm_args=["-Dauthentication.psk=ivan"])
s = Server(port=8080, jvm_args=["-DAuthHandlers=io.deephaven.auth.AnonymousAuthenticationHandler"])
s.start()

# Starting io.deephaven.python.server.EmbeddedServer
deephaven.cacheDir=C:\Users\Иван Кукушкин\AppData\Local\Deephaven Data Labs\deephaven\cache
deephaven.configDir=C:\Users\Иван Кукушкин\AppData\Roaming\Deephaven Data Labs\deephaven\config
deephaven.dataDir=C:\Users\Иван Кукушкин\AppData\Roaming\Deephaven Data Labs\deephaven\data
# io.deephaven.internal.log.LoggerFactoryServiceLoaderImpl: searching for 'io.deephaven.internal.log.LoggerFactory'...
# io.deephaven.internal.log.LoggerFactoryServiceLoaderImpl: found 'io.deephaven.internal.log.LoggerFactorySlf4j'
Server started on port 8080


In [2]:
import numpy as np
import random

import deephaven_server
from pydeephaven import Session
from deephaven.time import to_j_duration
#server = deephaven_server.Server(host="localhost", port=8080)
#sdh = Session(port=8080)
from deephaven import time_table
from deephaven import new_table
from deephaven.numpy import to_table
from deephaven.column import int_col
from deephaven.column import double_col

In [3]:
# Создаем массив углов от 0 до 355 включительно с шагом 5
angle_arr = np.arange(0, 356, 5)
# Переводим углы из градусов в радианы для вычисления синуса
sin_arr = np.sin(np.deg2rad(angle_arr))
# Объединяем массивы в переменную date_sin
date_sin = (angle_arr, sin_arr)
# Вывод результатов
#print("Массив углов:")
#print(angle_arr)
#print("\nМассив значений синусоиды:")
#print(sin_arr)

In [6]:
t_sin = new_table([double_col("X", sin_arr)])
display(t_sin)

deephaven.table.Table(io.deephaven.engine.table.Table(objectRef=0x000002163606B082, num_rows = 72, columns = {'X':...}))

In [ ]:
new_date = [random.uniform(-1, 1) for i in range(72)]
#print(new_date)

In [34]:
#t_sin = t_sin.update(formula=["X = new_date"])
new_date = [random.uniform(-1, 1) for i in range(72)]
t_sin = new_table([double_col("X", new_date)])

In [7]:
from deephaven import DynamicTableWriter
import deephaven.dtypes as dht

import random, string, threading, time

# Create a DynamicTableWriter with two columns: `A`(int) and `B`(String)
table_writer = DynamicTableWriter({"A": dht.int_, "B": dht.string})

result = table_writer.table


# Function to log data to the dynamic table
def thread_func():
    # for loop that defines how much data to populate to the table
    for i in range(10):
        # the data to put into the table
        a = random.randint(1, 100)
        b = random.choice(string.ascii_letters)

        # The write_row method adds a row to the table
        table_writer.write_row(a, b)

        # seconds between new rows inserted into the table
        time.sleep(1)


# Thread to log data to the dynamic table
thread = threading.Thread(target=thread_func)
thread.start()

In [17]:
from deephaven import DynamicTableWriter, time_table
import deephaven.dtypes as dht
import numpy as np
import threading
import time

# Функция для обновления данных в таблице
def update_table(writer, interval=0.2):
    while True:
        # Создаем новый массив значений синуса
        angle_arr = np.arange(0, 356, 5)
        sin_arr = np.sin(np.deg2rad(angle_arr))
        
        # Обновляем таблицу новыми значениями
        for sin_value in sin_arr:
            writer.write_row(sin_value)
        
        # Ожидаем заданный интервал времени перед следующим обновлением
        time.sleep(interval)

# Создаем DynamicTableWriter с одним столбцом 'X'
writer = DynamicTableWriter({"X": dht.double})

# Получаем динамическую таблицу из DynamicTableWriter
t_sin_dynamic = writer.table

# Запускаем фоновый поток для обновления таблицы
thread = threading.Thread(target=update_table, args=(writer,))
thread.start()
